In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)/'images'
fnames = get_image_files(path)
pat = r'/([^/]+)_\d+.*'
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
bs=64

pets = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
     get_items=get_image_files,
     splitter=RandomSplitter(),
     get_y=RegexLabeller(pat = r'/([^/]+)_\d+.*'),
     item_tfms=item_tfms,
     batch_tfms=batch_tfms
)
dls = pets.dataloaders(path, bs=bs)

In [3]:
learn = vision_learner(dls, "vit_tiny_patch16_224")

In [4]:
learn.fine_tune(1)

In [5]:
learn.export("exported_fastai")
learn.save("exported_model")

In [6]:
learn = load_learner("exported_fastai")

In [7]:
fname = fnames[0]; fname

In [8]:
learn.predict(fname)

In [ ]:
#| hide
from IPython.display import Markdown as md
def make_definition(word, pos, meaning, example):
    html = f'<h4 class="anchored" style="color:
    html += f'<p style="color:
    html += f'<p class="word-meaning" style="color:
    html += f'<p class="word-example" style="color:
    return md(html)

In [ ]:
#| echo: false
make_definition("Test DataLoader", "noun", "A fastai DataLoader which utilizes the validation transforms when sending data through the pipelines", "In deployment we utilize test dataloaders )

In [9]:
dl = learn.dls.test_dl([fname])

In [16]:
preds = learn.get_preds(dl=dl)[0]

In [17]:
softmax = preds.softmax(dim=1)
argmax = preds.argmax(dim=1)
labels = [learn.dls.vocab[pred] for pred in argmax]
softmax, argmax, labels

In [20]:
%%timeit
dl = learn.dls.test_dl([fname])
_ = learn.get_preds(dl=dl)

In [21]:
%%timeit
_ = learn.predict(fname)

In [36]:
learn.dls.cuda()

In [40]:
%%timeit
dl = learn.dls.test_dl([fname])
_ = learn.get_preds(dl=dl)

In [41]:
%%timeit
dl = learn.dls.test_dl([fname], num_workers=0)
_ = learn.get_preds(dl=dl)